<a href="https://colab.research.google.com/github/MK316/Myapps/blob/main/IPA_practice_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IPA Practice Application(0919)

# Without user name

In [ ]:
!pip install gradio

In [ ]:
#@markdown This app works, asking only once for each IPA
import pandas as pd
import random
import gradio as gr

# Load the CSV data from the provided URL
def load_data():
    url = "https://raw.githubusercontent.com/MK316/Myapps/main/data/IPA.csv"
    return pd.read_csv(url, encoding='utf-8')

df = load_data()  # Load data once at the start

# Properties to ask about for each IPA symbol
properties = ['Voicing', 'Place', 'Centrality', 'Oro-nasal', 'Manner']

# Track used IPA symbols to avoid repetition
used_ipa_symbols = []

def generate_question(current_ipa, current_property_index):
    ipa_symbol = current_ipa['IPA']
    property_name = properties[current_property_index]
    question = f"IPA Symbol: {ipa_symbol}\nWhat is the {property_name.lower()} of this IPA symbol?"
    answer = current_ipa[property_name].lower()
    return question, answer, property_name.lower()

def quiz_function(user_answer, score, trials, question_info, current_property_index):
    correct_answer = question_info['answer']
    property_name = question_info['property']

    # Check the answer
    if user_answer.lower() == correct_answer:
        score[0] += 1
        result = f"Correct! The {property_name} is '{correct_answer}'."
    else:
        result = f"Wrong! The correct {property_name} was '{correct_answer}'."

    # Increment the trial count
    trials[0] += 1

    # Update the property index to ask the next property
    current_property_index += 1

    # Check if we have asked all properties for the current IPA
    if current_property_index >= len(properties):
        # Select a new IPA symbol, ensuring it hasn't been used before
        if len(used_ipa_symbols) >= len(df):
            # All IPA symbols have been used, reset the used symbols list
            used_ipa_symbols.clear()

        # Pick a new IPA symbol that hasn't been used
        while True:
            current_ipa = df.sample(1).iloc[0]
            if current_ipa['IPA'] not in used_ipa_symbols:
                used_ipa_symbols.append(current_ipa['IPA'])  # Mark this IPA as used
                break

        current_property_index = 0  # Reset the property index for the new IPA
    else:
        # Continue asking for the next property of the same IPA
        current_ipa = question_info['current_ipa']

    # Generate the next question for the current or new IPA symbol
    new_question, new_answer, new_property = generate_question(current_ipa, current_property_index)
    question_info.update({
        'question': new_question,
        'answer': new_answer,
        'property': new_property,
        'current_ipa': current_ipa
    })

    # Return result, next question, current score/trials, and updated property index
    return f"{result} Total points: {score[0]}/{trials[0]}.", question_info['question'], current_property_index

def gradio_interface():
    score = [0]  # Track the user's score
    trials = [0]  # Track the number of trials (questions asked)
    current_property_index = [0]  # Track which property we are asking about
    question_info = {'question': '', 'answer': '', 'property': '', 'current_ipa': None}

    def initialize_quiz():
        # Select a new IPA symbol and mark it as used
        current_ipa = df.sample(1).iloc[0]
        used_ipa_symbols.append(current_ipa['IPA'])
        question_info['question'], question_info['answer'], question_info['property'] = generate_question(current_ipa, 0)
        question_info['current_ipa'] = current_ipa
        current_property_index[0] = 0  # Start from the first property
        return question_info['question'], "", current_property_index[0]

    def submit_answer(user_answer):
        result, question, updated_property_index = quiz_function(user_answer, score, trials, question_info, current_property_index[0])
        current_property_index[0] = updated_property_index  # Update the property index
        return result, question, ""

    def quit_quiz():
        # Return only one output to avoid displaying 'None'
        return f"Quiz ended. Your total score: {score[0]}/{trials[0]} points."

    # Now defining all the components inside the Blocks context properly
    with gr.Blocks() as app:
        with gr.Column():  # Column to ensure all elements are aligned vertically
            question_label = gr.Textbox(label="Question", interactive=False)
            answer_input = gr.Textbox(label="Your Answer", placeholder="Type your answer here")

            submit_button = gr.Button("Submit")  # Submit button comes right after the answer input

            output_label = gr.Textbox(label="Result", interactive=False)  # Result comes after the submit button

            quit_button = gr.Button("Quit")  # Quit button stays at the end

        # Handle the submit answer action
        submit_button.click(fn=submit_answer, inputs=[answer_input], outputs=[output_label, question_label, answer_input])

        # Handle the quit action (now only returning one output)
        quit_button.click(fn=quit_quiz, outputs=[output_label])

        # Initialize the first question
        question_label.value, _, _ = initialize_quiz()

    return app

app = gradio_interface()
app.launch()


# Variable issues: mixed user names when sharing †he session, and answer key mixup

In [ ]:
%%capture
!pip install gradio

# This is the final version (as of 0920)

In [ ]:
#@markdown Final version (Space)
import pandas as pd
import random
import gradio as gr

# Load the CSV data
def load_data():
    url = "https://raw.githubusercontent.com/MK316/Myapps/main/data/IPA.csv"
    try:
        data = pd.read_csv(url, encoding='utf-8')
        if 'IPA' not in data.columns:
            raise ValueError("The required 'IPA' column is missing in the data.")
        return data
    except Exception as e:
        raise ValueError(f"Error loading data: {e}")

df = load_data()

# Properties to ask about for each IPA symbol
properties = ['Voicing', 'Place', 'Centrality', 'Oro-nasal', 'Manner']

# Generate a new question
def generate_question(used_ipa_symbols):
    if df.empty or 'IPA' not in df.columns:
        raise ValueError("IPA data is not loaded properly or missing.")

    # Loop until we get a valid IPA symbol that hasn't been used yet
    while True:
        current_ipa = df.sample(1).iloc[0]
        if not pd.isnull(current_ipa['IPA']) and current_ipa['IPA'] not in used_ipa_symbols:
            available_properties = [prop for prop in properties if not pd.isnull(current_ipa[prop])]
            if available_properties:
                break

    used_ipa_symbols.append(current_ipa['IPA'])
    property_name = random.choice(available_properties)
    question = f"IPA Symbol: {current_ipa['IPA']}\nWhat is the {property_name.lower()} of this IPA symbol?"
    answer = current_ipa[property_name].lower()
    return question, answer, used_ipa_symbols

# Check the answer and update the score and trials
def quiz_function(user_answer, correct_answer, score, trials):
    if user_answer.lower() == correct_answer:
        score += 1
        result = f"Correct! The answer was '{correct_answer}'."
    else:
        result = f"Wrong! The correct answer was '{correct_answer}'."
    trials += 1
    return result, score, trials

# Define the Gradio interface
def gradio_interface():
    # Apply direct CSS to control button size and text rendering
    with gr.Blocks(css="""
        #start_button, #submit_button, #quit_button {
            width: 200px;
            height: 50px;
            font-size: 16px;
            color: black;
            background-color: #f0f0f0;
            text-align: center;
            padding: 0;
            display: inline-block;
            font-family: Arial, sans-serif;
            vertical-align: middle;
            line-height: 50px;
        }
    """) as app:
        # Define the components with `elem_id` to apply custom CSS
        name_input = gr.Textbox(label="Enter your name", placeholder="Your name")
        start_button = gr.Button("Start Quiz", elem_id="start_button")
        question_label = gr.Textbox(label="Question", interactive=False)
        answer_input = gr.Textbox(label="Your Answer", placeholder="Type your answer here")
        submit_button = gr.Button("Submit", elem_id="submit_button")
        output_label = gr.Textbox(label="Result", interactive=False)
        quit_button = gr.Button("Quit", elem_id="quit_button")

        # Initialize session data
        user_sessions = {}

        # Start quiz function
        def start_quiz(name):
            if not name:
                return gr.update(), "Please enter your name to start the quiz.", "", "", gr.update()
            # Initialize user session
            user_sessions[name] = {
                "score": 0,
                "trials": 0,
                "used_ipa_symbols": [],
                "current_answer": ""
            }
            # Generate first question
            question, answer, used_ipa_symbols = generate_question(user_sessions[name]["used_ipa_symbols"])
            user_sessions[name]["current_answer"] = answer
            user_sessions[name]["used_ipa_symbols"] = used_ipa_symbols
            return question, "", "", gr.update(value="Submit"), gr.update()

        # Submit answer function
        def submit_answer(name, user_answer):
            if name not in user_sessions:
                return gr.update(), "Please start the quiz first.", "", "", gr.update()

            session = user_sessions[name]
            # Check the answer
            result, score, trials = quiz_function(user_answer, session["current_answer"], session["score"], session["trials"])
            session["score"] = score
            session["trials"] = trials

            # Generate next question
            question, answer, used_ipa_symbols = generate_question(session["used_ipa_symbols"])
            session["current_answer"] = answer
            session["used_ipa_symbols"] = used_ipa_symbols

            # Ensure the submit button retains its label after being clicked
            return question, "", result, gr.update(value="Submit"), gr.update()

        # Quit quiz function
        def quit_quiz(name):
            if name not in user_sessions:
                return "You haven't started the quiz yet.", gr.update()

            session = user_sessions.pop(name)
            return f"Quiz ended. Well done, {name}! Your total score: {session['score']}/{session['trials']} points.", gr.update()

        # Bind functions to buttons
        start_button.click(
            fn=start_quiz,
            inputs=[name_input],
            outputs=[question_label, answer_input, output_label, submit_button, quit_button]
        )

        submit_button.click(
            fn=submit_answer,
            inputs=[name_input, answer_input],
            outputs=[question_label, answer_input, output_label, submit_button, quit_button]
        )

        quit_button.click(
            fn=quit_quiz,
            inputs=[name_input],
            outputs=[output_label, quit_button]
        )

    return app

app = gradio_interface()
app.launch()
